In [47]:
#coding=utf-8
import cPickle as pickle
import MySQLdb
import json
from sklearn.feature_extraction.text import TfidfTransformer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import numpy as np
import json



set_skill = pickle.load(file('dic\set_skill_v8_500.data'))

print len(set_skill)

list_skill = sorted(set_skill)

print sorted(set_skill)

db = MySQLdb.connect("10.120.26.46","yang","iiizb104","project" )
cursor = db.cursor()
# sql = "SELECT skill FROM project.job where jobno between 1 AND 1000"
sql = "SELECT jobno, skill FROM project.jobstorage WHERE skill NOT REGEXP '{}';"

# 执行SQL语句
cursor.execute(sql)
# 获取所有记录列表
results = cursor.fetchall()

list_of_lists = []
List_jobno = []

for row in results:
    tmpdic = json.loads(row[1])
    # for each in tmpdic:
    #     tmpdic[each]=1
    for each in set_skill:
        if each not in tmpdic:
            tmpdic[each]=0
    tmpList = []
    for each in list_skill:
        tmpList.append(tmpdic[each])
    # 空值陣列踢除
    if tmpList != np.zeros(len(set_skill)).tolist():
        list_of_lists.append(tmpList)
        List_jobno.append(row[0])

transformer = TfidfTransformer()
X = transformer.fit_transform(list_of_lists)

cluster_labels = KMeans(n_clusters=15).fit_predict(X)

# print cluster_labels


26
['.net', 'ajax', 'android', 'app', 'asp.net', 'c', 'c#', 'c#.net', 'c++', 'css', 'html', 'ios', 'java', 'javascript', 'jquery', 'jsp', 'linux', 'mysql', 'objective-c', 'php', 'python', 'sql', 'unix', 'visual basic', 'visual c++', 'xml']


In [7]:
db = MySQLdb.connect("10.120.26.46","yang","iiizb104","project" )
cursor = db.cursor()
for each in range(0, len(cluster_labels), 1):
    upsql = "UPDATE project.jobstorage SET cluster = %d WHERE jobno = %d" % (cluster_labels[each], List_jobno[each])
    try:
       cursor.execute(upsql)
       db.commit()
    except:
       db.rollback()

In [51]:
import pandas as pd
db = MySQLdb.connect("10.120.26.46","yang","iiizb104","project" )
cursor = db.cursor()

Kdf = pd.DataFrame(X.toarray(), columns=list_skill, index=List_jobno)
Kdf.columns.name = 'skill'
Kdf.index.name = 'jobno'

Kdf['cluster'] = cluster_labels

# print type([Kdf[Kdf['cluster'] == each]])

#將各群拆開獨立計算TFIDF向量平均
List_of_TFIDF = []
for each in range(0,15,1):
    tmpDF = Kdf[Kdf['cluster'] == each ]
    skillVector = (tmpDF[list_skill].sum(axis=0)/len(tmpDF.index)).tolist()
    List_of_TFIDF.append(skillVector)
    
for each in range(0,15,1):
    tmpJ = json.dumps(List_of_TFIDF[each], ensure_ascii=False)
    upsql = "UPDATE cluster SET clusterVector = '%s' where cID = %d" % (tmpJ, each)
    try:
       cursor.execute(upsql)
       db.commit()
    except:
       db.rollback()
db.close()
    